In [0]:
# -*- coding: utf-8 -*-

#########################################################################
############ CARGAR LAS LIBRERÍAS NECESARIAS ############################
#########################################################################

# Importar librerı́as necesarias
import numpy as np
import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils
# Importar modelos y capas que se van a usar
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization
# Importar el optimizador a usar
from keras.optimizers import SGD
# Importar el conjunto de datos
from keras.datasets import cifar100

# Import image proprocessors
from keras.preprocessing.image import ImageDataGenerator
# Import Early Stopping
from keras.callbacks import EarlyStopping
# TensorFlow no muestra warnings
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)

#########################################################################
######## FUNCIÓN PARA CARGAR Y MODIFICAR EL CONJUNTO DE DATOS ###########
#########################################################################

""" Sólo se le llama una vez. Devuelve 4 vectores conteniendo, por este orden, las imágenes de entrenamiento,
    las clases de las imagenes de entrenamiento, las imágenes del conjunto de test y las clases del conjunto de test.
"""
def cargarImagenes():
    # Cargamos Cifar100. Cada imagen tiene tamaño (32 , 32 , 3). Nos quedamos con las imágenes de 25 de las clases.
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    train_idx = np.isin(y_train, np.arange (25))
    train_idx = np.reshape(train_idx, -1)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]

    test_idx = np.isin(y_test, np.arange (25))
    test_idx = np.reshape(test_idx, -1)
    x_test = x_test[test_idx]
    y_test = y_test[test_idx]

    # Transformamos los vectores de clases en matrices. Cada componente se convierte en un vector de
    # ceros con un uno en la componente correspondiente a la clase a la que pertenece la imagen.
    y_train = np_utils.to_categorical(y_train, 25)
    y_test = np_utils.to_categorical(y_test, 25)

    return x_train, y_train, x_test, y_test

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

""" Devuelve el accuracy de un modelo (porcentaje de etiquetas bien predichas frente al total).
- labels: vector de etiquetas verdaderas.
- preds: vector de etiquetas predichas.
"""
def calcularAccuracy (labels, preds):
    labels = np.argmax(labels, axis = 1)
    preds = np.argmax(preds, axis = 1)

    accuracy = sum (labels == preds)/ len(labels)

    return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################


""" Pinta dos gráficas, una con la evolución de la función de pérdida en el conjunto de train y en el de validación,
    y otra con la evolución del accuracy en el conjunto de train y el de validación.
- hist: historial del entrenamiento del modelo (salida de fit () y fit_generator ()).
"""
def mostrarEvolucion(hist):
    loss = hist.history ['loss']
    val_loss = hist.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()

    acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    plt.plot(acc)
    plt.plot(val_acc )
    plt.legend(['Training accuracy', 'Validation accuracy'])
    plt.show()

#########################################################################
################## DEFINICIÓN DEL MODELO BASENET ########################
#########################################################################

""" Definición del modelo BaseNet del apartado 1.
- input_shape: dimesión de la entrada
"""
def model_baseNet(input_shape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(6, kernel_size=(5,5), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(16, kernel_size=(5,5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(25, activation='softmax'))
  return model

#########################################################################
######### DEFINICIÓN DEL OPTIMIZADOR Y COMPILACIÓN DEL MODELO ###########
#########################################################################

""" Definimos el optimizador y compilador del modelo.
- model: el modelo.
"""
def optimizadorCompilador(model):
  # Definimos la función pérdida a minimizar.Clasificación multiclase -> categorical_crossentropy
  # Argumento metrics las métricas que se quieren calcular a lo largo de todas las épocas.
  model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=SGD (lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True),
            metrics=['accuracy'])
  
  # Usar uno de los siguientes optimizadores:
  # optimizer=optimizer=keras.optimizers.Adadelta()
  # optimizer=SGD (lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
  
  # Teniendo el modelo base guardamos los pesos aleatorios con los que empieza la red, para poder
  # reestablecerlos después y comparar resultados entre no usar mejoras y sí usarlas.
  weights = model.get_weights()
  return weights

#########################################################################
###################### ENTRENAMIENTO DEL MODELO #########################
#########################################################################

""" Definimos el entrenamiento del modelo.
- model: el modelo.
- x_train: datos de entrenamiento.
- y_train: etiquetas de los datos de entrenamiento.
- datagen: nuestro ImageDataGenerator.
- batch_size: 
- ephocs: épocas.
- verbose: nos muestra información al hacer el ajuste.
"""
def train(model, x_train, y_train, datagen, batch_size=64, epochs=20, verbose=0):
  # Entrenamos el modelo con fit que recibe las imágenes de entrenamiento directamente.
  # histograma = model.fit(x_train, y_train,
  #      batch_size=batch_size,
  #      epochs=epochs,
  #      verbose=verbose,
  #      validation_data=(x_test, y_test))
  
  train_data = datagen.flow(x_train, y_train, batch_size=batch_size, subset='training')
  validation_data = datagen.flow(x_train, y_train, batch_size=batch_size, subset='validation')

  hist = model.fit_generator(train_data,
             steps_per_epoch = len(x_train)*0.9/batch_size,
             epochs = epochs,
             verbose = verbose,
             validation_data = validation_data,
             validation_steps = len (x_train)*0.1/batch_size)

  return hist

#########################################################################
################ PREDICCIÓN SOBRE EL CONJUNTO DE TEST ###################
#########################################################################

""" Predicción sobre el conjunto de test.
- model: el modelo usado.
- x_test: conjunto de test.
- y_test: etiquetas del conjunto de test.
"""
def prediccion(model, x_test, y_test):
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])


""" Ejercicio1 """
def ejercicio1():
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 64             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen = ImageDataGenerator(validation_split = 0.1)
  datagen.fit(x_train)

  model = model_baseNet(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  prediccion(model, x_test, y_test)

ejercicio1()

In [0]:
#########################################################################
########################## MEJORA DEL MODELO ############################
#########################################################################

# NORMALIZACIÓN. Utilice la clase ImageDataGenerator con los parámetros correctos para que los datos estén bien
# condicionados (media=0, std dev=1) para mejorar el entrenamiento.

""" Ejercicio2 - NORMALIZACIÓN """
def ejercicio2_normalizacion():
  print("NORMALIZACIÓN")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 32             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen_train, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

ejercicio2_normalizacion()

In [0]:
# AUMENTO DE DATOS. Usar parámetros de aumento de datos de la clase ImageDataGenerator, como zoom_range
# y/o horizontal_flip. No debería tener ningún aumento de datos en los conjuntos de validación ni test.

""" Ejercicio2 - AUMENTO DE DATOS. """
def ejercicio2_aumento():
  print("AUMENTO DE DATOS")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 64             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, zoom_range=0.7, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen_train, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

ejercicio2_aumento()

""" Ejercicio2_v2 - AUMENTO DE DATOS. """
def ejercicio2_aumento_v2():
  print("AUMENTO DE DATOS - V2")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 32             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
                                     vertical_flip=True, zoom_range=0.7, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen_train, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

#ejercicio2_aumento_v2()

In [0]:
# RED MÁS PROFUNDA. Experimente agregando más capas convolucionales y totalmente conectadas.
# Agregue más capas conv con canales de salida crecientes y también agregue más capas lineales (fc).
# No coloque una capa de maxpool después de cada capa conv ya que conduce a una pérdida excesiva de información.

""" Modelo baseNet primera mejora. """
def model_baseNet_mejora1(input_shape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(8, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(16, kernel_size=(5, 5), padding='same', activation='relu'))
  model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(75, activation='relu'))
  model.add(Dense(25, activation='softmax'))
  return model

def model_baseNet_mejora2(input_shape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(16, kernel_size=(5,5), activation='relu', input_shape=input_shape))
  #model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(4,4)))
  model.add(Dropout(0.3))
  model.add(Conv2D(8, kernel_size=(5,5), activation='relu'))
  #model.add(BatchNormalization())
  model.add(Conv2D(4, kernel_size=(3,3), activation='relu', padding = 'same'))
  #model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(25, activation='relu'))
  model.add(Dense(25, activation='softmax'))
  model.summary()
  return model

""" Ejercicio2 - RED MÁS PROFUNDA. """
def ejercicio2_red():
  print("RED MÁS PROFUNDA")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 32             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, zoom_range=0.7, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet_mejora1(input_shape)
  #model = model_baseNet_mejora2(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen_train, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

ejercicio2_red()

In [0]:
# CAPAS DE NORMALIZACIÓN. Las capas de normalización ayudan a reducir el sobreajuste y mejorar el entrenamiento
# del modelo. Las capas de normalización de Keras son una forma fácil de incorporarlas al modelo. Agregue capas de
# normalización después de las capas conv (BatchNormalization). Agregue capas de normalización después de capas lineales
# y experimente insertándolas antes o después de las capas ReLU.

""" Modelo baseNet primera mejora (BatchNormalization()). """
def model_baseNet_batch(input_shape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(BatchNormalization())
  model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(Conv2D(6, kernel_size=(3, 3), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(25, activation='softmax'))
  return model

""" Ejercicio2 - CAPAS DE NORMALIZACIÓN. """
def ejercicio2_capa_normalizacion():
  print("CAPAS DE NORMALIZACIÓN")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 32            # Número de clases es 25
  batch_size = 64             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, zoom_range=0.7, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet_batch(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  hist = train(model, x_train, y_train, datagen_train, batch_size, epochs, verbose=1)
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

ejercicio2_capa_normalizacion()

In [0]:
# "EARLY STOPPING". ¿Después de cuántas épocas parar y dejar de entrenar?

""" Ejercicio2 - EARLY STOPPING """
def ejercicio2_early_stopping():
  print("EARLY STOPPING")
  input_shape=(32, 32, 3)     # Imágenes en color con 3 canales de 32x32 píxeles.
  num_classes = 25            # Número de clases es 25
  batch_size = 32             # Tamaño de batch potencia de 2
  epochs = 20                 # Elegimos número de épocas

  x_train, y_train, x_test, y_test = cargarImagenes()
  datagen_train = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, horizontal_flip=True, zoom_range=0.7, validation_split=0.1)
  datagen_test = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
  datagen_train.fit(x_train)
  datagen_test.fit(x_train)

  model = model_baseNet_batch(input_shape)
  model.summary()             # Descripción del modelo
  weights = optimizadorCompilador(model)
  model.set_weights(weights)  # Reestablecemos los pesos

  train_data = datagen_train.flow(x_train, y_train, batch_size=batch_size, subset='training')
  validation_data = datagen_train.flow(x_train, y_train, batch_size=batch_size, subset='validation')
  es = EarlyStopping(monitor='val_loss', patience=5)
  hist = model.fit_generator(train_data,
            steps_per_epoch = len(x_train)*0.9/batch_size,
            epochs = epochs,
            verbose = 1,
            validation_data = validation_data,
            validation_steps = len (x_train)*0.1/batch_size,
            callbacks=[es])
  print(hist)
  mostrarEvolucion(hist)
  preds = model.predict_generator(datagen_test.flow(x_test, batch_size = 1, shuffle = False), steps = len(x_test))
  score = calcularAccuracy(y_test, preds)
  print('Test accuracy:', score)

ejercicio2_early_stopping()